In [142]:
#Запись в файл
data = {
    'test': Test
}
with open('test_set.pkl', 'wb') as output_file:
    pickle.dump(data, output_file, protocol=pickle.HIGHEST_PROTOCOL)

In [5]:
#Считывание из файла 
with open('minitest_set.pkl', 'rb') as input_file:
    data = pickle.load(input_file)

In [1]:
%%time
from sklearn import svm
from sklearn.feature_extraction.text import TfidfVectorizer
from itertools import groupby
from nltk import word_tokenize
from nltk.corpus import stopwords
import pandas as pd
import pickle
import nltk
import numpy as np
import math
import re
nltk.download('stopwords')

#Считываем файл, поправляя кодировку
Data = pd.read_csv('D:/загрузки/german_labeled.csv', sep=' ==> ', header=None, encoding = 'utf-8 sig')
#Получаем список тегов
Tags = [el for el, _ in groupby(sorted([str(tag) for tag in Data[4]]))]

#Инициализируем словарь предложений по тегам
Sentences_by_tag = {}
i=0
for Tag in Tags:    
    Sentences_by_tag[Tag] = i
    i=i+1

    
#Записываем все предложения по каждому тегу в словарь    
Sentences = ''
for Unique_tag in Sentences_by_tag.keys():
    i = 0
    Sentences_by_tag[Unique_tag] = []
    for Tag_to_compare in [str(tag) for tag in Data[4]]:         
        if Tag_to_compare == Unique_tag:
            Sentences += ' '+ str(Data[3][i])
        i=i+1
    Sentences_by_tag[Unique_tag] = Sentences
    Sentences = ''
    
#Получаем все предложения, отсортированные по тегам  
Documents = []
for Unique_tag in Sentences_by_tag.keys():
    Documents.append(Sentences_by_tag[Unique_tag])
for i in range(len(Documents)):
    Documents[i] = re.sub(r"\d+", "", Documents[i], flags=re.UNICODE)

#Получаем tf\idf матрицу с помощью sklearn    
vectorizer = TfidfVectorizer(lowercase=True, stop_words = stopwords.words('german'), ngram_range = (1, 2)) 
matrix = vectorizer.fit_transform(Documents)
word_list = vectorizer.get_feature_names()
amountWords = len(vectorizer.get_feature_names())  
Tf_idf=pd.DataFrame(matrix.toarray(), columns=word_list, index= Sentences_by_tag.keys())


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


D:\Anaconda\lib\site-packages\ipykernel_launcher.py:15: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  from ipykernel import kernelapp as app


Wall time: 15.6 s


In [2]:
def normalize(v):
    norm = np.linalg.norm(v)
    if norm == 0: 
        return v
    return v / norm

In [3]:
#Функция, превращающая предложение в список чисел в соответствии со схемой
def To_vector (Array_wsentences):
    stolbec = np.zeros(amountWords)
    for j in range(len(Array_wsentences)):
        if isinstance(Array_wsentences[j], str):
            splittedSentence = re.split(r"[,.!?:;\"\'\(\)\%\=\#\&\@\$\-\*\+\>\<\/\ \s]", Array_wsentences[j])
            while ("") in splittedSentence:
                splittedSentence.remove("")
            for word in splittedSentence:
                try:
                    position = word_list.index(word.lower())
                    if j == 0: 
                        stolbec[position] += 0.5
                    elif (j == 1) | (j == 2):
                        stolbec[position] += 0.333
                    elif (j == 3) | (j == 4):
                        stolbec[position] += 0.166
                except ValueError:                    
                    continue  
            if len(splittedSentence) > 1:
                for c in range (len(splittedSentence)-1):
                    pair = splittedSentence[c] + splittedSentence[c+1]
                    try:
                        position = word_list.index(pair.lower())
                        if j == 0: 
                            stolbec[position] += 0.5
                        elif (j == 1) | (j == 2):
                            stolbec[position] += 0.333
                        elif (j == 3) | (j == 4):
                            stolbec[position] += 0.166
                    except ValueError:                    
                        continue
        else:
            continue
    return(normalize(np.matmul(Tf_idf,stolbec)))

In [4]:
Training_data = []
for i in zip(Data[4],Data[3]):
    Training_data.append(i)

Test_data = Training_data[109743:]
Training_data = Training_data[:109743]
Minitest_data = Training_data[107743:]

In [6]:
#Превращаем тестовую выборку в вектора
def Create_vectors(Some_data):
    Test = []
    x = len(Some_data)
    for i in range(x):
        Temp = []
        if i+1 >= x: 
            Temp.append(Some_data[i][1])
            Temp.append(Some_data[i-1][1])
            Temp.append(Some_data[i-2][1])
        elif i+2 >= x:
            Temp.append(Some_data[i][1])
            Temp.append(Some_data[i-1][1])
            Temp.append(Some_data[i+1][1])
            Temp.append(Some_data[i-2][1])    
        else:
            Temp.append(Some_data[i][1])
            Temp.append(Some_data[i-1][1])
            Temp.append(Some_data[i+1][1])
            Temp.append(Some_data[i-2][1])  
            Temp.append(Some_data[i+2][1])
        print(i,'/',x)
        Test.append([Some_data[i][0], To_vector(Temp)]) 
    return(Test)

In [21]:
%%time
# Train = Create_vectors(Training_data)
# data = {
#     'train': Train
# }
# with open('train_set_bi.pkl', 'wb') as output_file:
#     pickle.dump(data, output_file, protocol=pickle.HIGHEST_PROTOCOL)
    
    
Test = Create_vectors(Test_data)
data = {
    'test': Test
}
with open('test_set_bi_3.pkl', 'wb') as output_file:
    pickle.dump(data, output_file, protocol=pickle.HIGHEST_PROTOCOL)
    
    
Minitest = Create_vectors(Minitest_data)
data = {
    'minitest': Minitest
}
with open('minitest_set_bi_3.pkl', 'wb') as output_file:
    pickle.dump(data, output_file, protocol=pickle.HIGHEST_PROTOCOL)

0 / 15678
1 / 15678
2 / 15678
3 / 15678
4 / 15678
5 / 15678
6 / 15678
7 / 15678
8 / 15678
9 / 15678
10 / 15678
11 / 15678
12 / 15678
13 / 15678
14 / 15678
15 / 15678
16 / 15678
17 / 15678
18 / 15678
19 / 15678
20 / 15678
21 / 15678
22 / 15678
23 / 15678
24 / 15678
25 / 15678
26 / 15678
27 / 15678
28 / 15678
29 / 15678
30 / 15678
31 / 15678
32 / 15678
33 / 15678
34 / 15678
35 / 15678
36 / 15678
37 / 15678
38 / 15678
39 / 15678
40 / 15678
41 / 15678
42 / 15678
43 / 15678
44 / 15678
45 / 15678
46 / 15678
47 / 15678
48 / 15678
49 / 15678
50 / 15678
51 / 15678
52 / 15678
53 / 15678
54 / 15678
55 / 15678
56 / 15678
57 / 15678
58 / 15678
59 / 15678
60 / 15678
61 / 15678
62 / 15678
63 / 15678
64 / 15678
65 / 15678
66 / 15678
67 / 15678
68 / 15678
69 / 15678
70 / 15678
71 / 15678
72 / 15678
73 / 15678
74 / 15678
75 / 15678
76 / 15678
77 / 15678
78 / 15678
79 / 15678
80 / 15678
81 / 15678
82 / 15678
83 / 15678
84 / 15678
85 / 15678
86 / 15678
87 / 15678
88 / 15678
89 / 15678
90 / 15678
91 / 1567

692 / 15678
693 / 15678
694 / 15678
695 / 15678
696 / 15678
697 / 15678
698 / 15678
699 / 15678
700 / 15678
701 / 15678
702 / 15678
703 / 15678
704 / 15678
705 / 15678
706 / 15678
707 / 15678
708 / 15678
709 / 15678
710 / 15678
711 / 15678
712 / 15678
713 / 15678
714 / 15678
715 / 15678
716 / 15678
717 / 15678
718 / 15678
719 / 15678
720 / 15678
721 / 15678
722 / 15678
723 / 15678
724 / 15678
725 / 15678
726 / 15678
727 / 15678
728 / 15678
729 / 15678
730 / 15678
731 / 15678
732 / 15678
733 / 15678
734 / 15678
735 / 15678
736 / 15678
737 / 15678
738 / 15678
739 / 15678
740 / 15678
741 / 15678
742 / 15678
743 / 15678
744 / 15678
745 / 15678
746 / 15678
747 / 15678
748 / 15678
749 / 15678
750 / 15678
751 / 15678
752 / 15678
753 / 15678
754 / 15678
755 / 15678
756 / 15678
757 / 15678
758 / 15678
759 / 15678
760 / 15678
761 / 15678
762 / 15678
763 / 15678
764 / 15678
765 / 15678
766 / 15678
767 / 15678
768 / 15678
769 / 15678
770 / 15678
771 / 15678
772 / 15678
773 / 15678
774 / 15678
775 

1346 / 15678
1347 / 15678
1348 / 15678
1349 / 15678
1350 / 15678
1351 / 15678
1352 / 15678
1353 / 15678
1354 / 15678
1355 / 15678
1356 / 15678
1357 / 15678
1358 / 15678
1359 / 15678
1360 / 15678
1361 / 15678
1362 / 15678
1363 / 15678
1364 / 15678
1365 / 15678
1366 / 15678
1367 / 15678
1368 / 15678
1369 / 15678
1370 / 15678
1371 / 15678
1372 / 15678
1373 / 15678
1374 / 15678
1375 / 15678
1376 / 15678
1377 / 15678
1378 / 15678
1379 / 15678
1380 / 15678
1381 / 15678
1382 / 15678
1383 / 15678
1384 / 15678
1385 / 15678
1386 / 15678
1387 / 15678
1388 / 15678
1389 / 15678
1390 / 15678
1391 / 15678
1392 / 15678
1393 / 15678
1394 / 15678
1395 / 15678
1396 / 15678
1397 / 15678
1398 / 15678
1399 / 15678
1400 / 15678
1401 / 15678
1402 / 15678
1403 / 15678
1404 / 15678
1405 / 15678
1406 / 15678
1407 / 15678
1408 / 15678
1409 / 15678
1410 / 15678
1411 / 15678
1412 / 15678
1413 / 15678
1414 / 15678
1415 / 15678
1416 / 15678
1417 / 15678
1418 / 15678
1419 / 15678
1420 / 15678
1421 / 15678
1422 / 15678

1977 / 15678
1978 / 15678
1979 / 15678
1980 / 15678
1981 / 15678
1982 / 15678
1983 / 15678
1984 / 15678
1985 / 15678
1986 / 15678
1987 / 15678
1988 / 15678
1989 / 15678
1990 / 15678
1991 / 15678
1992 / 15678
1993 / 15678
1994 / 15678
1995 / 15678
1996 / 15678
1997 / 15678
1998 / 15678
1999 / 15678
2000 / 15678
2001 / 15678
2002 / 15678
2003 / 15678
2004 / 15678
2005 / 15678
2006 / 15678
2007 / 15678
2008 / 15678
2009 / 15678
2010 / 15678
2011 / 15678
2012 / 15678
2013 / 15678
2014 / 15678
2015 / 15678
2016 / 15678
2017 / 15678
2018 / 15678
2019 / 15678
2020 / 15678
2021 / 15678
2022 / 15678
2023 / 15678
2024 / 15678
2025 / 15678
2026 / 15678
2027 / 15678
2028 / 15678
2029 / 15678
2030 / 15678
2031 / 15678
2032 / 15678
2033 / 15678
2034 / 15678
2035 / 15678
2036 / 15678
2037 / 15678
2038 / 15678
2039 / 15678
2040 / 15678
2041 / 15678
2042 / 15678
2043 / 15678
2044 / 15678
2045 / 15678
2046 / 15678
2047 / 15678
2048 / 15678
2049 / 15678
2050 / 15678
2051 / 15678
2052 / 15678
2053 / 15678

2608 / 15678
2609 / 15678
2610 / 15678
2611 / 15678
2612 / 15678
2613 / 15678
2614 / 15678
2615 / 15678
2616 / 15678
2617 / 15678
2618 / 15678
2619 / 15678
2620 / 15678
2621 / 15678
2622 / 15678
2623 / 15678
2624 / 15678
2625 / 15678
2626 / 15678
2627 / 15678
2628 / 15678
2629 / 15678
2630 / 15678
2631 / 15678
2632 / 15678
2633 / 15678
2634 / 15678
2635 / 15678
2636 / 15678
2637 / 15678
2638 / 15678
2639 / 15678
2640 / 15678
2641 / 15678
2642 / 15678
2643 / 15678
2644 / 15678
2645 / 15678
2646 / 15678
2647 / 15678
2648 / 15678
2649 / 15678
2650 / 15678
2651 / 15678
2652 / 15678
2653 / 15678
2654 / 15678
2655 / 15678
2656 / 15678
2657 / 15678
2658 / 15678
2659 / 15678
2660 / 15678
2661 / 15678
2662 / 15678
2663 / 15678
2664 / 15678
2665 / 15678
2666 / 15678
2667 / 15678
2668 / 15678
2669 / 15678
2670 / 15678
2671 / 15678
2672 / 15678
2673 / 15678
2674 / 15678
2675 / 15678
2676 / 15678
2677 / 15678
2678 / 15678
2679 / 15678
2680 / 15678
2681 / 15678
2682 / 15678
2683 / 15678
2684 / 15678

...

14855 / 15678
14856 / 15678
14857 / 15678
14858 / 15678
14859 / 15678
14860 / 15678
14861 / 15678
14862 / 15678
14863 / 15678
14864 / 15678
14865 / 15678
14866 / 15678
14867 / 15678
14868 / 15678
14869 / 15678
14870 / 15678
14871 / 15678
14872 / 15678
14873 / 15678
14874 / 15678
14875 / 15678
14876 / 15678
14877 / 15678
14878 / 15678
14879 / 15678
14880 / 15678
14881 / 15678
14882 / 15678
14883 / 15678
14884 / 15678
14885 / 15678
14886 / 15678
14887 / 15678
14888 / 15678
14889 / 15678
14890 / 15678
14891 / 15678
14892 / 15678
14893 / 15678
14894 / 15678
14895 / 15678
14896 / 15678
14897 / 15678
14898 / 15678
14899 / 15678
14900 / 15678
14901 / 15678
14902 / 15678
14903 / 15678
14904 / 15678
14905 / 15678
14906 / 15678
14907 / 15678
14908 / 15678
14909 / 15678
14910 / 15678
14911 / 15678
14912 / 15678
14913 / 15678
14914 / 15678
14915 / 15678
14916 / 15678
14917 / 15678
14918 / 15678
14919 / 15678
14920 / 15678
14921 / 15678
14922 / 15678
14923 / 15678
14924 / 15678
14925 / 15678
14926 

15441 / 15678
15442 / 15678
15443 / 15678
15444 / 15678
15445 / 15678
15446 / 15678
15447 / 15678
15448 / 15678
15449 / 15678
15450 / 15678
15451 / 15678
15452 / 15678
15453 / 15678
15454 / 15678
15455 / 15678
15456 / 15678
15457 / 15678
15458 / 15678
15459 / 15678
15460 / 15678
15461 / 15678
15462 / 15678
15463 / 15678
15464 / 15678
15465 / 15678
15466 / 15678
15467 / 15678
15468 / 15678
15469 / 15678
15470 / 15678
15471 / 15678
15472 / 15678
15473 / 15678
15474 / 15678
15475 / 15678
15476 / 15678
15477 / 15678
15478 / 15678
15479 / 15678
15480 / 15678
15481 / 15678
15482 / 15678
15483 / 15678
15484 / 15678
15485 / 15678
15486 / 15678
15487 / 15678
15488 / 15678
15489 / 15678
15490 / 15678
15491 / 15678
15492 / 15678
15493 / 15678
15494 / 15678
15495 / 15678
15496 / 15678
15497 / 15678
15498 / 15678
15499 / 15678
15500 / 15678
15501 / 15678
15502 / 15678
15503 / 15678
15504 / 15678
15505 / 15678
15506 / 15678
15507 / 15678
15508 / 15678
15509 / 15678
15510 / 15678
15511 / 15678
15512 

454 / 2000
455 / 2000
456 / 2000
457 / 2000
458 / 2000
459 / 2000
460 / 2000
461 / 2000
462 / 2000
463 / 2000
464 / 2000
465 / 2000
466 / 2000
467 / 2000
468 / 2000
469 / 2000
470 / 2000
471 / 2000
472 / 2000
473 / 2000
474 / 2000
475 / 2000
476 / 2000
477 / 2000
478 / 2000
479 / 2000
480 / 2000
481 / 2000
482 / 2000
483 / 2000
484 / 2000
485 / 2000
486 / 2000
487 / 2000
488 / 2000
489 / 2000
490 / 2000
491 / 2000
492 / 2000
493 / 2000
494 / 2000
495 / 2000
496 / 2000
497 / 2000
498 / 2000
499 / 2000
500 / 2000
501 / 2000
502 / 2000
503 / 2000
504 / 2000
505 / 2000
506 / 2000
507 / 2000
508 / 2000
509 / 2000
510 / 2000
511 / 2000
512 / 2000
513 / 2000
514 / 2000
515 / 2000
516 / 2000
517 / 2000
518 / 2000
519 / 2000
520 / 2000
521 / 2000
522 / 2000
523 / 2000
524 / 2000
525 / 2000
526 / 2000
527 / 2000
528 / 2000
529 / 2000
530 / 2000
531 / 2000
532 / 2000
533 / 2000
534 / 2000
535 / 2000
536 / 2000
537 / 2000
538 / 2000
539 / 2000
540 / 2000
541 / 2000
542 / 2000
543 / 2000
544 / 2000

1183 / 2000
1184 / 2000
1185 / 2000
1186 / 2000
1187 / 2000
1188 / 2000
1189 / 2000
1190 / 2000
1191 / 2000
1192 / 2000
1193 / 2000
1194 / 2000
1195 / 2000
1196 / 2000
1197 / 2000
1198 / 2000
1199 / 2000
1200 / 2000
1201 / 2000
1202 / 2000
1203 / 2000
1204 / 2000
1205 / 2000
1206 / 2000
1207 / 2000
1208 / 2000
1209 / 2000
1210 / 2000
1211 / 2000
1212 / 2000
1213 / 2000
1214 / 2000
1215 / 2000
1216 / 2000
1217 / 2000
1218 / 2000
1219 / 2000
1220 / 2000
1221 / 2000
1222 / 2000
1223 / 2000
1224 / 2000
1225 / 2000
1226 / 2000
1227 / 2000
1228 / 2000
1229 / 2000
1230 / 2000
1231 / 2000
1232 / 2000
1233 / 2000
1234 / 2000
1235 / 2000
1236 / 2000
1237 / 2000
1238 / 2000
1239 / 2000
1240 / 2000
1241 / 2000
1242 / 2000
1243 / 2000
1244 / 2000
1245 / 2000
1246 / 2000
1247 / 2000
1248 / 2000
1249 / 2000
1250 / 2000
1251 / 2000
1252 / 2000
1253 / 2000
1254 / 2000
1255 / 2000
1256 / 2000
1257 / 2000
1258 / 2000
1259 / 2000
1260 / 2000
1261 / 2000
1262 / 2000
1263 / 2000
1264 / 2000
1265 / 2000
1266

1866 / 2000
1867 / 2000
1868 / 2000
1869 / 2000
1870 / 2000
1871 / 2000
1872 / 2000
1873 / 2000
1874 / 2000
1875 / 2000
1876 / 2000
1877 / 2000
1878 / 2000
1879 / 2000
1880 / 2000
1881 / 2000
1882 / 2000
1883 / 2000
1884 / 2000
1885 / 2000
1886 / 2000
1887 / 2000
1888 / 2000
1889 / 2000
1890 / 2000
1891 / 2000
1892 / 2000
1893 / 2000
1894 / 2000
1895 / 2000
1896 / 2000
1897 / 2000
1898 / 2000
1899 / 2000
1900 / 2000
1901 / 2000
1902 / 2000
1903 / 2000
1904 / 2000
1905 / 2000
1906 / 2000
1907 / 2000
1908 / 2000
1909 / 2000
1910 / 2000
1911 / 2000
1912 / 2000
1913 / 2000
1914 / 2000
1915 / 2000
1916 / 2000
1917 / 2000
1918 / 2000
1919 / 2000
1920 / 2000
1921 / 2000
1922 / 2000
1923 / 2000
1924 / 2000
1925 / 2000
1926 / 2000
1927 / 2000
1928 / 2000
1929 / 2000
1930 / 2000
1931 / 2000
1932 / 2000
1933 / 2000
1934 / 2000
1935 / 2000
1936 / 2000
1937 / 2000
1938 / 2000
1939 / 2000
1940 / 2000
1941 / 2000
1942 / 2000
1943 / 2000
1944 / 2000
1945 / 2000
1946 / 2000
1947 / 2000
1948 / 2000
1949

In [40]:
with open('train_set_bi_1.pkl', 'rb') as input_file:
    data_learn = pickle.load(input_file)

    
with open('test_set_bi_1.pkl', 'rb') as input_file:
    data_test = pickle.load(input_file)

In [41]:
For_learn = []
For_learn = data_learn['train']

For_test = []
For_test = data_test['test']

In [42]:
Answers1 = []
Questions1 = []
for answer in For_learn:
    Answers1.append(answer[0])
    Questions1.append(answer[1])
    
Answers2 = []
Questions2 = []
for answer in For_test:
    Answers2.append(answer[0])
    Questions2.append(answer[1])

In [43]:
%%time
model = svm.SVC(kernel='linear')
model.fit(Questions1, Answers1)

Wall time: 16min 3s


In [44]:
%%time
model.score(Questions2, Answers2)

Wall time: 4min 13s


0.35954841178721775

In [39]:
i=0
for pair in For_test:
    if model.predict([pair[1]]) == pair[0]:
        #print(pair[0])
        i+=1
print(i/len(For_test))

D:\Anaconda\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  This is separate from the ipykernel package so we can avoid doing imports until


0.33671386656461283


In [ ]:
#Пока не работает\не нужно
#a = np.array(Sentences_by_tag.keys())
#b = np.array(Sentences_by_tag.values())
#c = np.array(x.loc["0"])

#
#Filtered_words = []
#for word in Documents:
#    Filtered_words+=re.split(r"[,.!?:;\"\'\(\)\%\=\#\&\@\$\-\*\+\>\<\/\ 1234567890\s]", str(word))
#Filtered_words=filter(None, Filtered_words)
#Unique_words = [el for el, _ in groupby(sorted(Filtered_words))]
#
#             ind = x.columns.index(word)
#             for feature in x.columns:
#                 if feature == word.lower():
#                     break
#                 position += 1